# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create a reference the CSV file desired
csv_path = "weather_data.csv"

# Read the CSV into a Pandas DataFrame
weather_df = pd.read_csv(csv_path)

# Print the first five rows of data to the screen
weather_df= weather_df.rename(columns={"Wind Speed":"Wind_Speed"})
weather_df.head()

,city,lat,lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,date
0,Faanui,-16.4833,-151.7500,27.30,75,37,6.26,PF,1619037877
1,Hithadhoo,-0.6000,73.0833,27.99,68,23,2.42,MV,1619037877
2,Rosita,13.9268,-84.4032,33.55,35,88,3.45,NI,1619037877
3,Hasaki,35.7333,140.8333,8.00,93,20,1.03,JP,1619037878
4,Cape Town,-33.9258,18.4232,19.00,77,0,3.60,ZA,1619038277


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["lat", "lng"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity , 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:

weather_check= weather_df.loc[(weather_df['Wind_Speed'] < 10) & (weather_df['Cloudiness'] ==0) & (weather_df['Max_Temp'] > 21.1) &(weather_df['Max_Temp'] < 26.7) ]
weather_check.count()
weather_final=weather_check.dropna()

weather_final.head()

,city,lat,lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,date
32,Hong Kong,22.2855,114.1577,22.78,82,0,2.91,HK,1619037624
150,São João da Barra,-21.6403,-41.0511,26.00,57,0,7.20,BR,1619037915
151,Saint-Philippe,-21.3585,55.7679,25.00,83,0,5.66,RE,1619037915
191,Assomada,15.1000,-23.6833,22.00,68,0,7.72,CV,1619037926
207,Guararapes,-21.2608,-50.6428,24.04,40,0,3.61,BR,1619037929


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
weather_final["Hotel Name"]=""
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in weather_final.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
#     print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        weather_final.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
weather_final

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,city,lat,lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,date,Hotel Name
32,Hong Kong,22.2855,114.1577,22.78,82,0,2.91,HK,1619037624,Royal Plaza Hotel 帝京酒店
150,São João da Barra,-21.6403,-41.0511,26.00,57,0,7.20,BR,1619037915,Tulip Inn Campos dos Goytacazes
151,Saint-Philippe,-21.3585,55.7679,25.00,83,0,5.66,RE,1619037915,Sud Hôtel
191,Assomada,15.1000,-23.6833,22.00,68,0,7.72,CV,1619037926,Hotel Oasis Atlantico Praiamar
207,Guararapes,-21.2608,-50.6428,24.04,40,0,3.61,BR,1619037929,Ipê Palace Hotel
225,Gunjur,13.2019,-16.7339,22.00,83,0,6.17,GM,1619037934,Kairaba Beach Hotel
285,Fort Abbas,29.1925,72.8536,25.96,32,0,3.74,PK,1619037949,Hotel Second Home
423,Mahrīz,31.5917,54.4318,25.00,15,0,3.09,IR,1619037984,
431,Le Port,-20.9373,55.2919,26.00,73,0,6.69,RE,1619037985,Hôtel & Spa Le Saint Alexis
445,Paranaíba,-19.6772,-51.1908,25.54,34,0,2.41,BR,1619037988,Thais Grandes Lagos Resort


In [8]:
hotel_df = weather_final[["Hotel Name","city","Country","lat","lng"]]

hotel_df

,Hotel Name,city,Country,lat,lng
32,Royal Plaza Hotel 帝京酒店,Hong Kong,HK,22.2855,114.1577
150,Tulip Inn Campos dos Goytacazes,São João da Barra,BR,-21.6403,-41.0511
151,Sud Hôtel,Saint-Philippe,RE,-21.3585,55.7679
191,Hotel Oasis Atlantico Praiamar,Assomada,CV,15.1000,-23.6833
207,Ipê Palace Hotel,Guararapes,BR,-21.2608,-50.6428
225,Kairaba Beach Hotel,Gunjur,GM,13.2019,-16.7339
285,Hotel Second Home,Fort Abbas,PK,29.1925,72.8536
423,,Mahrīz,IR,31.5917,54.4318
431,Hôtel & Spa Le Saint Alexis,Le Port,RE,-20.9373,55.2919
445,Thais Grandes Lagos Resort,Paranaíba,BR,-19.6772,-51.1908


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [16]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))

In [17]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))